In [ ]:
# pip install sentence-transformers scikit-learn numpy
from sentence_transformers import SentenceTransformer
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity

# 1) Canonical tags
beauty_accs = [
    'Хорошо пенится',
    'Деликатно очищает',
    'Удобный',
    'Отличный состав',
    'Для ночного ухода',
    'Высокая плотность',
    'Питает кожу',
    'Удлиняет ресницы',
    'Сужает поры',
    'Быстро расходуется',
    'Нежный',
    'Легкий аромат',
    'Не оставляет пленки',
    'Освежает лицо',
    'Не стягивает кожу',
    'Слабое отшелушивание',
    'Совместим с кремами',
    'Для сухих волос',
    'Выраженный эффект',
    'Для чувствительной кожи',
    'Не сушит',
    'Освежает дыхание',
    'Отличное качество',
    'Подсушивает',
    'Мягко очищает',
    'Даёт объём',
    'Увлажняет надолго',
]

len(beauty_accs)

# Чтобы не изменять bag_tags везде, передадим так
bag_tags = beauty_accs

# 3) загрузка модели (можно заменить на другую модель)
model = SentenceTransformer('deepvk/user-bge-m3')

# 4) вычисляем эмбеддинги canonical тегов (делаем один раз)
canon_emb = model.encode(bag_tags, convert_to_numpy=True, normalize_embeddings=True)

# 5) функция маппинга: возвращает top_k кандидатов и только те с similarity >= threshold
def map_raw_tags_with_embeddings(raw_tags, bag_tags, canon_emb, model,
                                 threshold=0.65, top_k=1, batch_size=32):
    """
    raw_tags: iterable of raw tag strings
    bag_tags: list canonical tags (same order as canon_emb)
    canon_emb: numpy array of canonical embeddings (L2-normalized)
    model: sentence-transformers model
    threshold: cosine similarity threshold (0..1)
    top_k: how many top candidates to return per raw tag
    returns: dict raw_tag -> list of (canonical_tag, similarity) sorted desc
    """
    results = {}
    # encode in batches
    for i in range(0, len(raw_tags), batch_size):
        batch = raw_tags[i:i+batch_size]
        emb = model.encode(batch, convert_to_numpy=True, normalize_embeddings=True)
        # cosine similarity matrix: (batch_size x n_canon)
        sims = np.dot(emb, canon_emb.T)  # because normalized, dot == cosine
        for j, raw in enumerate(batch):
            row = sims[j]
            # top_k indices
            idxs = np.argsort(-row)[:top_k]
            candidates = []
            for idx in idxs:
                score = float(row[idx])
                if score >= threshold:
                    candidates.append((bag_tags[idx], score))
            results[raw] = candidates  # empty list if nothing passed threshold
    return results




/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/195 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/54.0 [00:00<?, ?B/s]

...

tokenizer_config.json: 0.00B [00:00, ?B/s]

sentencepiece.bpe.model:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/963 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/297 [00:00<?, ?B/s]

In [ ]:
import pandas as pd

!gdown 1wwu3XmdLz7jQ5avywdLzKJkEEVkg7tru

# Имеющиеся

tags = pd.read_csv('/content/lamoda_reviews_sampled_with_tags.csv')
tags = tags[tags['tags'].notnull()]

Downloading...
From: https://drive.google.com/uc?id=1wwu3XmdLz7jQ5avywdLzKJkEEVkg7tru
To: /content/lamoda_reviews_sampled_with_tags.csv
100% 2.88M/2.88M [00:00<00:00, 224MB/s]


In [ ]:
# Новые

beauty_tags = pd.read_csv('/content/beauty_llm_tags_1000_skus.csv')
beauty_tags = beauty_tags.rename(columns={'llm_tags': 'tags'})

beauty_tags

,product_sku,comment_text,tags
0,AC001LMDWBR8,"Запах кайф, текстура кайф\nприятно пахнет, увл...",Плотная; приятный запах; рыхлая; сушит кожу; л...
1,AC001LWJOYF0,Отличная шутка . Жесткая . Но хватает надолго ...,Жесткая; Мягкая; Хорошо чистит; С чувствительн...
2,AN042LWGLC72,"Красивый цвет. Хорошие лаки, этот не первый и ...",Быстро сохнет; Долго держится; Ровно ложится; ...
3,AN042LWGLD12,"Использую как базу и как финиш. Сохнет быстро,...",Глянцевое покрытие; быстро сохнет; укрепляет н...
4,AN042LWIWK72,"Хороший лак, к покупки рекомендую\nОтличный ла...",Быстро сохнет; Ровно ложится; Долго держится; ...
...,...,...,...
1207,XD001XW01780,"Пришло с отломанным замочком. Цвет бледный, вы...",Замочок отломан; Цвет бледный; Надписи нет; Ка...
1208,XD001XW01CCS,потрясный аромат\nНе навязчивый стойкий\nАрома...,Нежный; стеклянный; сирень; уютный; тёплый; лё...
1209,XD001XW01CRL,"Очень деликатное нанесение , без комков , не с...",Без комков; Не склеивает; Уходовое действие; Н...
1210,XD001XW01JAT,"Запах приятный, пока тестирую\nДавно хотела по...",Приятный запах; Качественная упаковка; Свежест...


In [ ]:
cat_tags = tags[tags['good_type'] == 'Beauty_Accs']
cat_tags

,product_sku,comment_id,comment_text,name,good_type,good_subtype,tags
2,MP002XW0TITY,"[429537120, 429326163, 431280940, 397131465, 4...","[""Хороший крем"", ""Крем понравился, хорошо увла...",Крем для рук,Beauty_Accs,HANDS CARE,Хорошо увлажняет; Питает; Впитывается быстро; ...
13,RTLABC980601,"[383763719, 379895635, 378283967, 382568580, 4...","[""Приятный запах, не липкое"", ""Губы увлажненны...",Масло для губ,Beauty_Accs,LIPS CARE,Не липкое; Увлажняет; Насыщенный цвет; Приятны...
18,MP002XW1C416,"[400201013, 417074349, 384972549, 429347734, 3...","[""Отличный скраб, запах очень приятный, спасиб...",Скраб для тела,Beauty_Accs,BATH & SHOWER,Приятный аромат; Увлажняет кожу; Ненавязчивый ...
19,RTLACB593301,"[430416845, 422198645, 424541459, 392123166, 3...","[""Оригинал! Цвет огонь!"", ""Классный блеск, пря...",Бальзам оттеночный для губ,Beauty_Accs,MAKE-UP LIPS,Нейтральный оттенок; Перламутровый оттенок; По...
22,MP002XU0D7UV,"[413634621, 389281977, 379533936, 429321862, 3...","[""Запах восхитительный. Средства свои функции ...",Набор для ухода за руками,Beauty_Accs,HANDS CARE,Увлажняет хорошо; Быстро впитывается; Приятная...
...,...,...,...,...,...,...,...
2636,MP002XW0AROG,"[394188599, 390274771, 384279905, 384441378]","[""Сыворотка очень нравиться!"", ""Брала в подаро...",Сыворотка для лица,Beauty_Accs,FACE CARE,Хорошо увлажняет; Высокое качество; Оригинал
2644,MP002XW10ML2,"[404522261, 378421858, 402992446, 420558774, 4...","[""Отличнейший спрей. Наношу на слегка влажные ...",Спрей для волос,Beauty_Accs,HAIR CARE,Приятный аромат; Увлажняет волосы; Облегчает р...
2645,MP002XU002DN,"[419739807, 407815173, 411530156, 399642221, 4...","[""люблю более выраженный вкус"", ""Пришло хорошо...",Ополаскиватель для полости рта,Beauty_Accs,ORAL CARE,Концентрат; Освежающий; Не пенится; Приятный в...
2646,MP002XW0D54O,"[378278645, 394993163, 381289517, 410214761]","[""Отличный крем, но вместо авокадо пришел бана...",Набор для ухода за руками,Beauty_Accs,HANDS CARE,Быстро впитывается; Не оставляет жирности; Не ...


In [ ]:
# Конкат

tags = (
    pd.concat([cat_tags, beauty_tags], ignore_index=True)
      .drop_duplicates()
)
tags = tags.drop(columns = ['comment_id', 'name', 'good_type', 'good_subtype'])

In [ ]:
dataset = []

for _, item in tags.iterrows():
    tag_list = [tag.strip() for tag in str(item['tags']).split(';') if tag.strip()]
    dataset.append((item['comment_text'], tag_list))


In [ ]:
dataset[0][1]

['Хорошо увлажняет',
 'Питает',
 'Впитывается быстро',
 'Не липкий',
 'Приятный аромат',
 'Не навязчивый аромат',
 'Не жирный',
 'Смягчает',
 'Для лета',
 'Большой объем',
 'Парфюмированный']

In [ ]:
dataset[0][0]

'["Хороший крем", "Крем понравился, хорошо увлажняет и питает, впитывается быстро", "Запах приятный ,хорошо впитывается  ", "Крем замечательный, впитывается быстро, не липкий, рекомендую", "Неплохой крем. Не жирный. Смягчает,  но не увлажняет. Для лета хорошо,  но для зимы нужно пожирнее.", " Большой объем, хорошо увлажняет, приятный не навязчивый аромат.", "Очень парфюмированный и  эффект не понравился"]'

In [ ]:
len(dataset)

1831

In [ ]:
# пример использования

final = []

for i in range(len(dataset)):
    # Берем raw tag датасета
    raw_tags = dataset[i][1]
    mapped = map_raw_tags_with_embeddings(raw_tags, bag_tags, canon_emb, model,
                                        threshold=0.7, top_k=1)

    tags_to_send = []
    for r, cands in mapped.items():
        tag = cands[0][0] if cands else None
        if tag != None:
            tags_to_send.append(tag)
        print(r, "->", cands, tag)

    if tags_to_send:
        final.append((dataset[i][0], list(set(tags_to_send))))
    print()

Выходные данные были обрезаны до нескольких последних строк (5000).
Приятный аромат -> [('Легкий аромат', 0.8724750280380249)] Легкий аромат
Хорошее масло -> [] None
Подходит для баночного массажа -> [] None
Дренажный массаж -> [] None

Легкий -> [('Удобный', 0.723291277885437)] Удобный
Увлажняет -> [('Увлажняет надолго', 0.8851032257080078)] Увлажняет надолго
Без пленки -> [] None
Подходит для сухой кожи -> [('Для сухих волос', 0.8075948357582092)] Для сухих волос
Пользуется часто -> [] None

Быстро впитывается -> [('Быстро расходуется', 0.7235442399978638)] Быстро расходуется
Уменьшает черные точки -> [('Мягко очищает', 0.7162291407585144)] Мягко очищает
Подсушивает воспаления -> [('Подсушивает', 0.7453091144561768)] Подсушивает

Мягкие волосы -> [('Для сухих волос', 0.7207173109054565)] Для сухих волос
Хорошо пенится -> [('Хорошо пенится', 0.9999998807907104)] Хорошо пенится
Без химии -> [] None
Не вызывает зуда -> [] None
Неприятный аромат -> [] None
Появилась перхоть -> [] None
Пл

In [ ]:
for i in range(10):
    print(final[i][1])

['Питает кожу', 'Мягко очищает', 'Увлажняет надолго', 'Не стягивает кожу', 'Высокая плотность', 'Легкий аромат', 'Быстро расходуется']
['Подсушивает', 'Увлажняет надолго', 'Не стягивает кожу', 'Удобный', 'Легкий аромат']
['Мягко очищает', 'Легкий аромат', 'Увлажняет надолго', 'Быстро расходуется']
['Легкий аромат', 'Увлажняет надолго', 'Не стягивает кожу']
['Удобный', 'Легкий аромат', 'Увлажняет надолго', 'Быстро расходуется']
['Мягко очищает', 'Даёт объём', 'Увлажняет надолго', 'Быстро расходуется', 'Не сушит']
['Освежает лицо', 'Легкий аромат']
['Мягко очищает', 'Легкий аромат', 'Удлиняет ресницы']
['Подсушивает', 'Для сухих волос', 'Отличное качество']
['Освежает лицо']


In [ ]:
final[0]

('Стильная, приятная по ощущениям.\nТёплый удобный\nХорошая, рекомендую\nОчень колючая\nОтлично смотрится. заказал вторую такую же.\nКлассево выглядит, состав шикарный!\nХорошая, лёгкая шапка. Качество на уровне\n',
 ['Приятная ткань', 'Держит тепло', 'Легко стирается', 'Качественный'])

In [ ]:
len(final)

1793

In [ ]:
import csv
import json

name = "beauty_dataset1800.csv"

with open(name, "w", newline="", encoding="utf-8-sig") as f:
    writer = csv.writer(f)
    writer.writerow(["text", "tags"])  # заголовки

    for text, tags in final:
        writer.writerow([text, json.dumps(tags, ensure_ascii=False)])


In [ ]:
bags500 = pd.read_csv(name)
bags500

,text,tags
0,"[""Хороший крем"", ""Крем понравился, хорошо увла...","[""Питает кожу"", ""Мягко очищает"", ""Увлажняет на..."
1,"[""Приятный запах, не липкое"", ""Губы увлажненны...","[""Подсушивает"", ""Увлажняет надолго"", ""Не стяги..."
2,"[""Отличный скраб, запах очень приятный, спасиб...","[""Мягко очищает"", ""Легкий аромат"", ""Увлажняет ..."
3,"[""Оригинал! Цвет огонь!"", ""Классный блеск, пря...","[""Легкий аромат"", ""Увлажняет надолго"", ""Не стя..."
4,"[""Запах восхитительный. Средства свои функции ...","[""Удобный"", ""Легкий аромат"", ""Увлажняет надолг..."
...,...,...
1788,"Пришло с отломанным замочком. Цвет бледный, вы...","[""Отличное качество""]"
1789,потрясный аромат\nНе навязчивый стойкий\nАрома...,"[""Удобный"", ""Нежный""]"
1790,"Очень деликатное нанесение , без комков , не с...","[""Не сушит"", ""Не стягивает кожу""]"
1791,"Запах приятный, пока тестирую\nДавно хотела по...","[""Освежает лицо"", ""Легкий аромат"", ""Отличное к..."
